# Baselines

In this notebook we explore some baselines for the problem of question answering. 

In [ ]:
import os
import numpy as np
import pandas as pd
import math
import spacy
import benepar
import json
from tqdm import tqdm

import utils

# Download spacy corpora of text in case it's needed
!python -m spacy download en_core_web_sm
# Download the benepar neural constituency parser
benepar.download('benepar_en3')

Load the training dataset and create a DataFrame containing:
- The paragraph's text
- The question's text
- The questions's ID

In [2]:
dataset = utils.read_question_set(os.path.join('..','data','training_set.json'))

# Create a more useful data structure using list comprehensions
questions = pd.DataFrame([{
        'context': paragraph['context'],
        'question': qa['question'],
        'questionID': qa['id'],
    }   for article in dataset['data']
        for paragraph in article['paragraphs']
        for qa in paragraph['qas'] ])

In [3]:
print(len(questions))
display(questions.head(5))

87599


,context,question,questionID
0,"Architecturally, the school has a Catholic cha...",To whom did the Virgin Mary allegedly appear i...,5733be284776f41900661182
1,"Architecturally, the school has a Catholic cha...",What is in front of the Notre Dame Main Building?,5733be284776f4190066117f
2,"Architecturally, the school has a Catholic cha...",The Basilica of the Sacred heart at Notre Dame...,5733be284776f41900661180
3,"Architecturally, the school has a Catholic cha...",What is the Grotto at Notre Dame?,5733be284776f41900661181
4,"Architecturally, the school has a Catholic cha...",What sits on top of the Main Building at Notre...,5733be284776f4190066117e


We define a method that, given the question's ID and the start and end probabilities generated by the baselines, returns the span of text containing the prediction.

In [5]:
def get_predictions(baseline, limit=None):
    predictions = {}

    limit = range(limit) if limit is not None else range(len(questions))

    for q in tqdm(limit):
        # Obtain start and end probabilities from the baseline function
        pstartv, pendv = baseline(
           questions.iloc[q,:]
        )
        # Obtain the indices of the best answer
        start, end = utils.start_end_token_from_probabilities(
            pstartv, pendv, dim=pstartv.shape[1]
        )[0]
        # Add the ID-answer pair in the predictions dictionary
        id = questions['questionID'].iloc[q]
        text = questions['context'].iloc[q]
        predictions[id] = text[start:end]
    
    return predictions


## 1. Random prediction baseline

We implement a predictor that returns random start and end probabilities. Then, we use the function `start_end_token_from_probabilities` to obtain the max-scoring randomly generated span of text. 

In [6]:
def random_baseline_predict(data):
    '''
    Creates random prediction vectors.
    '''
    pstartv = np.random.random((1, len(data['context'])))
    pendv = np.random.random((1, len(data['context'])))
    return pstartv, pendv

In [13]:
with open(os.path.join('eval','random_predictions.txt'), 'w') as f:
    json.dump(get_predictions(random_baseline_predict), f)

## 2. Sliding window baseline

The sliding window baseline is implemented in the same way it was presented in the
SQuAD v1 paper and in the [MCTest paper](https://aclanthology.org/D13-1020.pdf) by Richardson et al.
which originally proposed it.

Apart from the paragraph and the question, the implementation also needs a set of candidate answers.
SQuAD's paper proposes to "*only use spans which are constituents in the constituency parse generated by
Stanford CoreNLP*". In our case, we use a neural parser: **Berkeley Neural Parser**, which is the option
proposed by the `spacy` library, which we are already using as a named entity extractor in the original model.

In [14]:
def C(w, P):
    # Sum of all words in the sentence where the word is equal to w
    return sum([1 for p in P if w == p])

def IC(w, P):
    # A scoring function based on inverse word frequency
    return math.log(1 + (1/C(w, P)))

def sliding_window_baseline_predict(data):
    # Initialize spacy's pipeline which we'll use for analysis
    spacy_pipeline = spacy.load("en_core_web_sm")
    # Step 1: Create a set of words present in the question (always ignore punctuation)
    Q = set([token for token in spacy_pipeline(data['question']) if not token.is_punct])
    # Step 2: use benepar to produce a constituency tree
    spacy_pipeline.add_pipe('benepar', config={'model': 'benepar_en3'})
    # Process the text using Spacy's pipeline
    doc = spacy_pipeline(data['context'])
    # Obtain a list of tokens in the paragraph
    paragraph_tokens = [token for token in doc]
    P = [token for token in paragraph_tokens if not token.is_punct]
    # From the processed text we can also obtain the list of constituents,
    # which will be the proposed answers to each question. We also create
    # a set of words of the question.
    proposed_answers = [ {
            'answer': answer,
            'start': answer.start_char,
            'end': answer.end_char,
            'token_set': set(tok for tok in answer if not tok.is_punct)
        }
        for sentence in list(doc.sents)         # Iterate over sentences
        for answer in list(sentence._.children) # Iterate over constitents
        if not len(set(tok for tok in answer if not tok.is_punct)) == 0     # Ignore span if it only contains punctuation
    ]

    # Step 3: Now that we have the question's text, the proposed answers and the context,
    # we can apply the sliding window algorithm, which computes a score based on the n-gram
    # overlap between the question's words and the proposed spans of text.
    scores = []
    for i in range(len(proposed_answers)):     # Iterate over all possible answers
        S = set([str(s) for s in proposed_answers[i]['token_set'].union(Q)])  # Unite the question and the answer words
        sj = []                         # Prepare a list for the scores
        for j in range(len(P)):         # For each word in the passage
            sx = 0  
            for w in range(len(S)):     # For each word in question/answer set
                try:
                    if str(P[j+w]) in S:     # Try to compute the score of this span
                        sx += IC(str(P[j+w]), [str(p) for p in P])    # And add it to the sum
                except IndexError:
                    pass
            sj.append(sx)
        sw = max(sj)                   # Eventually, select the max scoring  
        scores.append(sw)

    pstartv = np.zeros((1, len(data['context'])))
    pendv = np.zeros((1, len(data['context'])))
    best_scoring_answer = proposed_answers[np.argmax(scores)]
    pstartv[0, best_scoring_answer['start']] = 1
    pendv[0, best_scoring_answer['end']] = 1

    return pstartv, pendv

In [15]:
with open(os.path.join('eval','sliding_predictions.txt'), 'w') as f:
    json.dump(get_predictions(sliding_window_baseline_predict), f)

C:\Users\Volpe\Anaconda3\envs\NLP\lib\site-packages\torch\distributions\distribution.py:44: UserWarning: <class 'torch_struct.distributions.TreeCRF'> does not define `arg_constraints`. Please set `arg_constraints = {}` or initialize the distribution with `validate_args=False` to turn off validation.
  warnings.warn(f'{self.__class__} does not define `arg_constraints`. ' +
